## My inital attept at trying to run through a Kaggle competition
Found https://www.kaggle.com/c/web-traffic-time-series-forecasting/kernels?sortBy=votes&after=false


## Setup

In [1]:
from theano.sandbox import cuda

from imp import reload
%matplotlib inline
import utils; reload(utils)
from utils import *
from __future__ import division, print_function
from random import randint

import numpy as np
import h5py
import pandas as pd
import os, os.path
import math

WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: GeForce GTX 1080 (CNMeM is disabled, cuDNN 5110)
Using Theano backend.


In [2]:
get_ipython().magic('load_ext cellevents')

In [3]:
# No sample
#path = "/home/jd/data/fish/sample/"
path = "/home/jd/data/webTraffic/"
batch_size=64

time: 953 µs


## Load Data

In [4]:
key = pd.read_csv(path+'key_1.csv')
train = pd.read_csv(path+'train_1.csv')
submission = pd.read_csv(path+'sample_submission_1.csv')

time: 21.3 s


In [121]:
key

,Page,Id
0,!vote_en.wikipedia.org_all-access_all-agents_2...,bf4edcf969af
1,!vote_en.wikipedia.org_all-access_all-agents_2...,929ed2bf52b9
2,!vote_en.wikipedia.org_all-access_all-agents_2...,ff29d0f51d5c
3,!vote_en.wikipedia.org_all-access_all-agents_2...,e98873359be6
4,!vote_en.wikipedia.org_all-access_all-agents_2...,fa012434263a
5,!vote_en.wikipedia.org_all-access_all-agents_2...,48f1e93517a2
6,!vote_en.wikipedia.org_all-access_all-agents_2...,5def418fcb36
7,!vote_en.wikipedia.org_all-access_all-agents_2...,77bd08134351
8,!vote_en.wikipedia.org_all-access_all-agents_2...,5889e6dbb16f
9,!vote_en.wikipedia.org_all-access_all-agents_2...,5f21fef1d764


time: 10.1 ms


,Id,Visits
0,bf4edcf969af,0
1,929ed2bf52b9,0
2,ff29d0f51d5c,0
3,e98873359be6,0
4,fa012434263a,0
5,48f1e93517a2,0
6,5def418fcb36,0
7,77bd08134351,0
8,5889e6dbb16f,0
9,5f21fef1d764,0


time: 9.81 ms


In [106]:
(list(train.Page)).index('2NE1_zh.wikipedia.org_all-access_spider')

0

time: 6.98 ms


In [105]:
checktit.index('2NE1_zh.wikipedia.org_all-access_spider')

0

time: 2.85 ms


## Get the Weekdays

In [6]:
column_dates=list(train[0:0])

time: 908 µs


In [7]:
from datetime import date
import calendar
my_date = date.today()
calendar.day_name[my_date.weekday()]

'Friday'

time: 3.66 ms


In [8]:
import datetime

time: 514 µs


In [9]:
def todayofweek(i):
    year, month, day = (int(x) for x in i.split('-'))    
    answer = datetime.date(year, month, day).weekday()
    #dateValue.DayOfWeek
    #return (calendar.day_name[answer])
    return (answer)
    

time: 3.55 ms


In [10]:
for i in column_dates[1:10]:
    print (todayofweek(i))

2
3
4
5
6
0
1
2
3
time: 2.15 ms


## Calculate Daily Averages

In [11]:
traindrop=train.drop('Page', axis=1)

time: 98.9 ms


In [12]:
#traindrop

time: 481 µs


In [13]:
## Creates an array for the weekdays
WeekArray=[0,0,0,0,0,0,0]

time: 810 µs


In [14]:
## Gets the number of Weekdays in the dataset
for i in traindrop[:1]:
    WeekArray[todayofweek(i)]=WeekArray[todayofweek(i)]+1

time: 3.24 ms


In [15]:
print(WeekArray)

[78, 78, 79, 79, 79, 79, 78]
time: 1 ms


#### reset and try to add the first set

In [16]:
WeekArray=[0,0,0,0,0,0,0]
WeekArrayCount=[0,0,0,0,0,0,0]

time: 1.05 ms


In [17]:
trains=traindrop[0:1].values[:,:]

time: 2.94 ms


In [18]:
trains=list(trains[0])

time: 777 µs


In [19]:
trains[0]

18.0

time: 1.73 ms


In [20]:
## Gets the number of Weekdays in the dataset
count = 0
for i in traindrop[:1]:
    WeekArray[todayofweek(i)]=WeekArray[todayofweek(i)]+trains[count]
    WeekArrayCount[todayofweek(i)]=WeekArrayCount[todayofweek(i)]+1
    count=count+1

time: 6.52 ms


In [21]:
print (WeekArray)
print (WeekArrayCount)


for i in range (0,7):
    WeekArray[i]=math.ceil(WeekArray[i]/WeekArrayCount[i])

[1376.0, 1931.0, 1707.0, 1576.0, 1732.0, 1810.0, 1834.0]
[78, 78, 79, 79, 79, 79, 78]
time: 2.43 ms


In [22]:
datetimes=list(traindrop[:0])

time: 975 µs


In [23]:
datetimes[0]

'2015-07-01'

time: 1.67 ms


In [24]:
## prints out the average for the day   
print (WeekArray)

[18, 25, 22, 20, 22, 23, 24]
time: 875 µs


In [25]:
#train

time: 479 µs


## Averages for all

In [26]:
valuesforall=traindrop[:].values

time: 997 µs


In [27]:
errors=float(valuesforall[4][4])

time: 4.59 ms


In [28]:
np.isnan(valuesforall[0][4])

False

time: 1.79 ms


In [29]:
if "nan" == errors:
    print ("Hi")

time: 837 µs


In [30]:
## Creates an array for the weekdays
dailyaverage, pages = 7, 145062;
DailyAveragesArray = [[0 for x in range(dailyaverage)] for y in range(pages)] 
DailyAveragesArray[8][6]

0

time: 311 ms


In [31]:
datetimes[1]

'2015-07-02'

time: 1.47 ms


In [32]:
WeekArrayCount=[0,0,0,0,0,0,0]

time: 740 µs


In [36]:
for i in range(0,145061): #145061
    if i%10000 == 0:
        print (i/10000)
    WeekArrayCount=[0,0,0,0,0,0,0]
    for s in range(0,len(valuesforall[1])):
        if (np.isnan(valuesforall[i][s])):
            break
        else:
            DailyAveragesArray[i][todayofweek(datetimes[s])]=DailyAveragesArray[i][todayofweek(datetimes[s])]+valuesforall[i][s]
            WeekArrayCount[todayofweek(datetimes[s])]=WeekArrayCount[todayofweek(datetimes[s])]+1
    for s in range(0,7):
        if(WeekArrayCount[s]==0):
            DailyAveragesArray[i][s]=0
        else:
            DailyAveragesArray[i][s]=math.ceil(DailyAveragesArray[i][s]/WeekArrayCount[s])
        

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
time: 8min 18s


In [42]:
DailyAveragesArray[14]

[39, 42, 38, 36, 48, 48, 43]

time: 1.52 ms


In [51]:
# I want to save this!
import numpy
numpy.save(path+"weights",DailyAveragesArray)

time: 71.9 ms


In [117]:
DailyAveragesArray = numpy.load(path+"weights.npy")

time: 3.09 ms


In [118]:
DailyAveragesArray[37206][6]

4

time: 1.61 ms


### Putting it back in

In [93]:
key.Id[0]

'bf4edcf969af'

time: 1.7 ms


In [144]:
len(key.Page)

8703780

time: 1.51 ms


In [ ]:
namer = list()
visiter = list()

In [ ]:
namer = backupnamer
visiter = backupvisiter

for i in range(2264820,8703779): #8703779 or len(key.Page) -1
    if i%10000 == 0:
        print (i/10000)
    name=(key.Page[i])[:len(key.Page[i])-11]
    date=(key.Page[i])[len(key.Page[i])-10:]
    
    #print (name+" "+date)
    #print (key.Id[i])
    
    try:
        out=DailyAveragesArray[(list(train.Page)).index(name)][todayofweek(date)]
    except:
        out = -1
        print ("exception at " + str(len(visiter)))

    namer.append(str(key.Id[i]))
    visiter.append(str(out))

exception at2264820
exception at2264821
exception at2264822
exception at2264823
exception at2264824
exception at2264825
exception at2264826
exception at2264827
exception at2264828
exception at2264829
exception at2264830
exception at2264831
exception at2264832
exception at2264833
exception at2264834
exception at2264835
exception at2264836
exception at2264837
exception at2264838
exception at2264839
exception at2264840
exception at2264841
exception at2264842
exception at2264843
exception at2264844
exception at2264845
exception at2264846
exception at2264847
exception at2264848
exception at2264849
exception at2264850
exception at2264851
exception at2264852
exception at2264853
exception at2264854
exception at2264855
exception at2264856
exception at2264857
exception at2264858
exception at2264859
exception at2264860
exception at2264861
exception at2264862
exception at2264863
exception at2264864
exception at2264865
exception at2264866
exception at2264867
exception at2264868
exception at2264869


In [150]:
# save a copy at 2264820
backupvisiter=visiter
backupnamer=namer

time: 828 µs


In [149]:
len(visiter)

2264820

time: 1.48 ms


In [141]:
subm = pd.DataFrame({'Id': namer, 'Visits': visiter})

time: 1.3 ms


In [142]:
subm

,Id,Visits
0,bf4edcf969af,4
1,929ed2bf52b9,2
2,ff29d0f51d5c,3
3,e98873359be6,5
4,fa012434263a,3


time: 4.04 ms


## Goof around with numbers

In [56]:
len(submission)

8703780

time: 1.58 ms


In [57]:
# Lists all of the ids
ids = key.Id.values
# lists all of the pages
pages = key.Page.values

d_pages = {}
# d_pages has the id and the page name
for id, page in zip(ids, pages):
    d_pages[id] = page[:-11]

time: 4.01 s


In [58]:
pages = train.Page.values

time: 708 µs


In [59]:
# list of all the wikipedia pages
pages[1]

'2PM_zh.wikipedia.org_all-access_spider'

time: 7.08 ms


In [60]:
visits = np.nan_to_num(np.round(np.nanmedian(train.drop('Page', axis=1).values[:, -56:], axis=1)))

time: 502 ms


/home/jd/.local/lib/python3.5/site-packages/numpy/lib/function_base.py:3858: RuntimeWarning: All-NaN slice encountered
  r = func(a, **kwargs)


In [62]:
visits

array([ 18.,  19.,   4., ...,   0.,   0.,   0.])

time: 1.97 ms


In [61]:
d_visits = {}
for page, visits_number in zip(pages, visits):
    d_visits[page] = visits_number

time: 45.1 ms


In [ ]:
print('Modifying sample submission...')
ss_ids = submission.Id.values
ss_visits = submission.Visits.values


In [ ]:
## Assings a value to each result
for i, ss_id in enumerate(ss_ids):
    ss_visits[i] = (d_visits[d_pages[ss_id]])


In [ ]:
subm = pd.DataFrame({'Id': ss_ids, 'Visits': ss_visits})

In [ ]:
print (subm)

## Submit

In [ ]:
# Makes the submission name out of folders
numFiles=len(os.listdir(path+'submits/'))
subm_name = 'submission'+str(numFiles)+'.csv.gz'
submission=subm

In [ ]:
submission.to_csv((path+'submits/'+subm_name), index=False, compression='gzip')

In [ ]:
#FileLink('/data/webTraffic/submits/'+subm_name)

In [ ]:
print ('http://localhost:8888/view/data/webTraffic/submits/'+subm_name)